In [4]:
import cupy as cp
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import statistics as st
import optuna

from sklearn.linear_model import Ridge
from sklearn.feature_selection import f_regression
from plotly.subplots import make_subplots
from scipy.stats import weibull_min
from sklearn import linear_model
from datetime import datetime, timedelta

In [5]:
df1 = pd.read_csv("Trend01_20221101.csv")
df2 = pd.read_csv("Trend01_20221102.csv")
df3 = pd.read_csv("Trend01_20221103.csv")
df4 = pd.read_csv("Trend01_20221104.csv")
df5 = pd.read_csv("Trend01_20221105.csv")
df6 = pd.read_csv("Trend01_20221106.csv")
df7 = pd.read_csv("Trend01_20221107.csv")

df8  = pd.read_csv("Trend01_20221108.csv")
df9  = pd.read_csv("Trend01_20221109.csv")
df10 = pd.read_csv("Trend01_20221110.csv")
df11 = pd.read_csv("Trend01_20221111.csv")
df12 = pd.read_csv("Trend01_20221112.csv")
df13 = pd.read_csv("Trend01_20221113.csv")
df14 = pd.read_csv("Trend01_20221114.csv")

In [6]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7,
                df8, df9, df10, df11, df12, df13, df14])

# df.plot()
time = df['F01'].to_numpy()


In [90]:
Time_A = df11['F01'].to_numpy()
Time_proj = df12["F01"].to_numpy  # TimeProj(t) = t + 1

TCUI2_temp_A = cp.array(df11["PVI_T2031"])

torque_EX1_A = df11["PVI_T2097"]
torque_EX2_A = df11["PVI_T2052"]



In [91]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=0)

features = ['PVI_T2097', 'PVI_T2052', 'PVI_T2031']
target = 'PVI_T2031'

dtrain = xgb.DMatrix(train_df[features], label=train_df[target])
dtest = xgb.DMatrix(test_df[features], label=test_df[target])


# Set up the XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'max_depth': 5,
    'eta': 0.1,
    'eval_metric': 'rmse'
}

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=100)


#Predict on 20% test data
predictions = model.predict(dtest)

print(predictions)
print(df['PVI_T2031'].to_numpy())

predictions = model.predict(dtest)
actuals = test_df[target].to_numpy()
rmse = np.sqrt(np.mean((predictions - actuals)**2))
print("RMSE:", rmse)




[189.72333 190.50824 207.22836 ... 226.11023 223.53198 227.52663]
[190.8 190.8 190.8 ... 188.4 188.4 188.4]
RMSE: 0.053154066459805416


In [92]:
dtarget = xgb.DMatrix(df[features], label=df[target])
prediction_new = model.predict(dtarget)

print(len(prediction_new))

241904


In [93]:
print(len(predictions))
print(len(df['PVI_T2031'].to_numpy()))

48381
241904


In [94]:
targetX =  np.array(df['PVI_T2031'].to_numpy())

In [ ]:
fig = px.line(x= time, y = prediction_new)
fig.add_scatter(x = time, y = targetX, mode = 'lines')
fig

In [96]:
#Time future projection

delta = pd.to_datetime(df["F01"])
last_time = df['F01'].max()


# Calculate the next day's time
next_day = last_time + timedelta(days=1)

# Create a new row for the next day's data
new_row = pd.DataFrame({'time': [next_day],
                        'sensor_1': [0],
                        'sensor_2': [0],
                        'sensor_3': [0],
                        # ...
                       })



SyntaxError: ':' expected after dictionary key (1882442527.py, line 15)

In [ ]:
import optuna
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the objective function for Optuna to minimize
def objective(trial):
    # Set the XGBoost hyperparameters for this trial
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    }
    
    # Train an XGBoost model using the set of hyperparameters
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Return the negative accuracy so that Optuna minimizes this value
    return -accuracy

# Create an Optuna study
study = optuna.create_study(direction="maximize")

# Run the optimization
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params

In [ ]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7,
                df8, df9, df10, df11, df12, df13, df14])

time = df['F01'].to_numpy()

train_df, test_df = train_test_split(df, test_size=0.2, random_state=0)

features = ['PVI_T2097', 'PVI_T2052', 'PVI_T2031']
target = 'PVI_T2031'

dtrain = xgb.DMatrix(train_df[features], label=train_df[target])
dtest = xgb.DMatrix(test_df[features], label=test_df[target])

#optuna optimisation loop
def objective(trial):
    params = {
        'objective': 'reg:squarederror',
        'max_depth': trial.suggest_int('max_depth', 2, 8),
        'eta': trial.suggest_uniform('eta', 0.05, 0.3),
    }

    #xgboost train
    model = xgb.train(params, dtrain, num_boost_round=100)
    predictions = model.predict(dtest)
    actuals = test_df[target].to_numpy()
    rmse = np.sqrt(np.mean((predictions - actuals)**2))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)


# Use the best hyperparameters from Optuna to train a final model on all the training data
best_params = study.best_params
best_model = xgb.train(best_params, xgb.DMatrix(df[features], label=df[target]), num_boost_round=100)




In [ ]:
dat = xgb.DMatrix(df[features], label=df[target])

# Use the final model for prediction on new data
new_data = dat # data that need to make predictions on

# new_dmatrix = xgb.DMatrix(new_data)
new_predictions = best_model.predict(dat)



